In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [2]:
import glob
import os

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm, trange

In [3]:
TEST_PATH = '/content/drive/MyDrive/abnormal-distribution-project-data/fin-BERT_HS/test'
TRAIN_PATH = '/content/drive/MyDrive/abnormal-distribution-project-data/fin-BERT_HS/train'
VALID_PATH = '/content/drive/MyDrive/abnormal-distribution-project-data/fin-BERT_HS/valid'

In [4]:
def read_csv(path):
    df = pd.read_csv(path)
    return df


def correct_data(df):
    col = df.columns
    df[col[1]] = df[col[1]].str.strip("b\'\"")
    df[col[2]] = df[col[2]].str.strip("b\'\"")
    df2 = df[col[5]].str.strip("tf.Tensor(\n[], shape=(768,), dtype=float32)").str.split().apply(pd.Series).astype(np.float32).add_prefix("HS_")
    df.drop(col[5], axis=1, inplace=True)
    df[col[6]] = df[col[6]].str.strip("tf.Tensor(\n[], shape=(768,), dtype=float32)").astype(np.float32)
    df[col[7]] = df[col[7]].str.strip("tf.Tensor(\n[], shape=(768,), dtype=float32)").astype(np.float32)
    df[col[8]] = df[col[8]].str.strip("tf.Tensor(\n[], shape=(768,), dtype=float32)").astype(np.float32)
    df = pd.concat([df, df2], axis=1, sort=False)
    return df
 
   
def get_files(path):
    files = glob.glob(path+'/*.csv')
    return files


def get_concat_df(df_list):
    df = pd.concat(df_list, axis=0)
    return df


def save_df (df, batch, path):
    path = path + '/consolidated_{:05d}.csv'
    path = path.format(batch)
    df.to_csv(path, index=False)


def delete_files(files):
    for file in files:
        os.remove(file)

def run_correction(path):
    files = get_files(path)
    df_list = []
    for i, file in tqdm(enumerate(files)):
        i += 1
        df = read_csv(file)
        df_list.append(correct_data(df))
        if i % 1000 == 0:
            df = get_concat_df(df_list)
            save_df(df, i//1000-1, path)
            df_list = []
    
    if df_list != []:
        df = get_concat_df(df_list)
        save_df(df, i//1000, path)
        df_list = []

    return files

In [5]:
files = run_correction(TRAIN_PATH)

In [6]:
delete_files(files)